In [93]:
import requests
import pandas as pd
import random
from bs4 import BeautifulSoup
import math
import time
import os
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import FormulaRule
from openpyxl.styles import Font, Alignment, Border, Side

In [2]:
def get_with_backoff(url, max_retries=6, timeout=20):
    """
    Retries on 429 (rate limit) with exponential backoff + jitter.
    Respects Retry-After header when present.
    Also retries on 5xx and some network errors.
    """
    for attempt in range(max_retries):
        try:
            r = session.get(url, timeout=timeout)

            # Debug (optional)
            print(r.status_code, r.headers.get("Retry-After"))

            # Handle rate limiting
            if r.status_code == 429:
                retry_after = r.headers.get("Retry-After")
                if retry_after:
                    # Retry-After can be seconds. If it's a date, this may fail -> fallback.
                    try:
                        wait = float(retry_after)
                    except ValueError:
                        wait = (2 ** attempt) + random.uniform(0.5, 1.5)
                else:
                    wait = (2 ** attempt) + random.uniform(0.5, 1.5)

                print(f"429 rate-limited. Sleeping {wait:.2f}s then retrying... ({attempt+1}/{max_retries})")
                time.sleep(wait)
                continue

            # Retry transient server errors
            if 500 <= r.status_code < 600:
                wait = (2 ** attempt) + random.uniform(0.5, 1.5)
                print(f"{r.status_code} server error. Sleeping {wait:.2f}s then retrying... ({attempt+1}/{max_retries})")
                time.sleep(wait)
                continue

            # Raise for other non-200s (403, 404, etc.)
            r.raise_for_status()
            return r

        except (requests.exceptions.Timeout,
                requests.exceptions.ConnectionError,
                requests.exceptions.ChunkedEncodingError) as e:
            wait = (2 ** attempt) + random.uniform(0.5, 1.5)
            print(f"Network error: {e}. Sleeping {wait:.2f}s then retrying... ({attempt+1}/{max_retries})")
            time.sleep(wait)

    raise RuntimeError(f"Failed after {max_retries} retries: {url}")

In [105]:
# Data that the user will be able to change
jobTitle = "Junior Python"
location = "Spain"
# Limit it to 1 type
workType = "en remoto"
# WiLL default to 10
numberOfJobs = 20

In [106]:
job_id_list = set()
jobTitleFormatted = jobTitle.replace(' ', "%20")

pageNumber = 0
pageNumberObj =  math.ceil(numberOfJobs / 10)
numberResults = ""

wt_map = {
    "en remoto": "2",
    "hibrido": "3",
    "presencial": "1"
}
wt_value = wt_map.get(workType)
WT = f"f_WT={wt_value}"

while pageNumber < pageNumberObj:
    # Form the link
    jobs_list_link = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={jobTitleFormatted}&location={location}&{WT}{numberResults}"
    print(jobs_list_link)
    # Save the response in text and format it to get all job posts
    response_jlg = requests.get(jobs_list_link)
    jobs_list_data = response_jlg.text
    jobs_list_soup = BeautifulSoup(jobs_list_data, "html.parser")
    jobs_page = jobs_list_soup.find_all("li")

    # Get all job ids from the posts
    for job in jobs_page:
        base_card_div = job.find("div", {"class": "base-card"})
        job_id = base_card_div.get("data-entity-urn").split(":")[3]
        job_id_list.add(job_id)

    pageNumber += 1
    numberResults = f"&start={pageNumber * 10}" 
# Debug
print(job_id_list)

https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Junior%20Python&location=Spain&f_WT=2
https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Junior%20Python&location=Spain&f_WT=2&start=10
{'4375730080', '4344868693', '4371175870', '4344099336', '4344753207', '4345824234', '4368612475', '4343901472', '4373999796', '4376580751', '4377139056', '4344139135', '4342452527', '4272412678', '4344988192', '4370527191', '4348296606', '4344898728', '4344958263', '4225591133'}


In [107]:
jobs_list = []
session = requests.Session()

for job_id in job_id_list:
    job_especifics_link = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"

    try:
        response_je = get_with_backoff(job_especifics_link, max_retries=6)
    except Exception as e:
        print(f"Skipping job_id {job_id} due to request failure: {e}")
        continue

    job_soup = BeautifulSoup(response_je.text, "html.parser")
    job_info = {"job_id": job_id}

    # Job Title (raise if missing)
    tag = job_soup.select_one("a.topcard__link h2.top-card-layout__title")
    if not tag:
        print(f"Missing job title for job_id {job_id} — skipping")
        continue
    job_info["job_title"] = tag.get_text(strip=True)
    
    # Company Info (raise if missing)
    tag = job_soup.find("a", class_="topcard__org-name-link")
    if not tag:
        print(f"Missing company tag for job_id {job_id} — skipping")
        continue
    job_info["company_name"] = tag.get_text(strip=True)
    job_info["company_link"] = f'=HYPERLINK("{tag.get("href")}","Link")' 
    
    # Workplace access (optional)
    job_info["workplace_access"] = workType

    # Apply link 
    tag = job_soup.select_one("div.top-card-layout__entity-info a.topcard__link")
    job_info["apply_link"] = f'=HYPERLINK("{tag.get("href")}","Link")'  

    # Job Type (optional)
    tag = job_soup.select_one("span.posted-time-ago__text")
    job_info["work_type"] = tag.get_text(strip=True) if tag else None

    # Job Level and wory schedule (optional)
    criteria_items = job_soup.select("li.description__job-criteria-item")
    
    job_info["seniority_level"] = None
    job_info["employment_type"] = None
    
    for item in criteria_items:
        header = item.select_one("h3.description__job-criteria-subheader")
        value = item.select_one("span.description__job-criteria-text")
    
        if not header or not value:
            continue
    
        header_text = header.get_text(strip=True)
    
        if header_text == "Seniority level":
            job_info["seniority_level"] = value.get_text(strip=True)
    
        elif header_text == "Employment type":
            job_info["employment_type"] = value.get_text(strip=True)
        
    # Number of applicants (optional)
    tag = job_soup.select_one("figcaption.num-applicants__caption")
    job_info["num_applicants"] = tag.get_text(strip=True) if tag else None

    # Time posted (optional)
    tag = job_soup.select_one("span.posted-time-ago__text")
    job_info["time_posted"] = tag.get_text(strip=True) if tag else None




    jobs_list.append(job_info)

    # Gentle pacing between job detail requests (helps reduce 429s)
    time.sleep(random.uniform(1.0, 2.5))

print("Scraping ended")

200 None
200 None
200 None
200 None
200 None
200 None
200 None
200 None
200 None
200 None
200 None
200 None
200 None
200 None
200 None
200 None
200 None
200 None
200 None
200 None
Scraping ended


In [108]:
jobs_df = pd.DataFrame(jobs_list)
#jobs_df

In [109]:
def format_jobhunt_excel(FILE):
    """Run ONCE when the Excel is first created: Estado + dropdown + colors + header style."""
    wb = load_workbook(FILE)
    ws = wb.active

    STATUS_HEADER = "Estado"
    options = ["A aplicar", "Aplicado", "En revisión", "Aceptado", "Rechazado"]
    default_value = "A aplicar"

    fills = {
        "A aplicar": PatternFill(start_color='FFFFF2CC', end_color='FFFFF2CC', fill_type='solid'),  # amarillo
        "Aplicado": PatternFill(start_color='FFD9E1F2', end_color='FFD9E1F2', fill_type='solid'),   # azul
        "En revisión": PatternFill(start_color='FFE2EFDA', end_color='FFE2EFDA', fill_type='solid'),# verde claro
        "Aceptado": PatternFill(start_color='FFC6EFCE', end_color='FFC6EFCE', fill_type='solid'),   # verde fuerte
        "Rechazado": PatternFill(start_color='FFFFC7CE', end_color='FFFFC7CE', fill_type='solid'),  # rojo
    }

    headers = [ws.cell(row=1, column=c).value for c in range(1, ws.max_column + 1)]

    # Create Estado only if missing (so it won't mess with your existing file)
    if STATUS_HEADER not in headers:
        ws.insert_cols(2)
        status_col = 2
        ws.cell(row=1, column=status_col, value=STATUS_HEADER)

        # default values
        for r in range(2, ws.max_row + 1):
            ws.cell(row=r, column=status_col, value=default_value)

        col_letter = ws.cell(row=1, column=status_col).column_letter
        rng = f"{col_letter}2:{col_letter}10000"  # covers future rows too

        # dropdown
        dv = DataValidation(type="list", formula1=f'"{",".join(options)}"', allow_blank=False)
        ws.add_data_validation(dv)
        dv.add(rng)

        # conditional colors
        for value, fill in fills.items():
            rule = FormulaRule(formula=[f'${col_letter}2="{value}"'], fill=fill)
            ws.conditional_formatting.add(rng, rule)

    # Header style (safe to re-run, but we call it once)
    bold_font = Font(bold=True)
    center_align = Alignment(horizontal="center", vertical="center")

    black_side = Side(style="thin", color="FF000000")
    border = Border(left=black_side, right=black_side, top=black_side, bottom=black_side)

    for col in range(1, ws.max_column + 1):
        cell = ws.cell(row=1, column=col)
        cell.font = bold_font
        cell.alignment = center_align
        cell.border = border

    ws.freeze_panes = "A2"
    wb.save(FILE)


def auto_resize_columns(FILE, min_w=10, max_w=85, padding=2):
    """Run EVERY time after writing the Excel: auto-fit column widths."""
    wb = load_workbook(FILE)
    ws = wb.active

    for col_cells in ws.columns:
        col_letter = col_cells[0].column_letter
        max_len = 0

        for cell in col_cells:
            value = cell.value
            if value is None:
                continue

            # If you use =HYPERLINK("url","Link"), display is "Link"
            if isinstance(value, str) and value.startswith("=HYPERLINK"):
                display = "Link"
            else:
                display = str(value)

            max_len = max(max_len, len(display))

        new_width = min(max_w, max(min_w, max_len + padding))
        ws.column_dimensions[col_letter].width = new_width

    wb.save(FILE)


def append_new_jobs_preserve_format(FILE, jobs_df, id_col="job_id", sheet_name=None, default_estado="A aplicar"):
    """
    Appends only NEW rows (by id_col) to an existing formatted Excel, preserving all formatting.
    Assumes headers are on row 1.
    """
    wb = load_workbook(FILE)
    ws = wb[sheet_name] if sheet_name else wb.active

    # Read headers from Excel
    headers = [ws.cell(row=1, column=c).value for c in range(1, ws.max_column + 1)]
    header_to_col = {h: i+1 for i, h in enumerate(headers) if h is not None}

    if id_col not in header_to_col:
        raise ValueError(f"'{id_col}' no está en el Excel. Headers actuales: {headers}")

    # Collect existing IDs from Excel (fast enough for normal sizes)
    id_excel_col = header_to_col[id_col]
    existing_ids = set()
    for r in range(2, ws.max_row + 1):
        v = ws.cell(row=r, column=id_excel_col).value
        if v is not None and str(v).strip() != "":
            existing_ids.add(str(v))

    # Ensure jobs_df id is str
    df = jobs_df.copy()
    df[id_col] = df[id_col].astype(str)

    # Filter only new rows
    new_df = df[~df[id_col].isin(existing_ids)]
    if new_df.empty:
        return 0  # nothing appended

    # If Estado column exists in Excel, ensure we can write default for new rows
    estado_col = header_to_col.get("Estado", None)

    # Append rows at bottom
    start_row = ws.max_row + 1

    # We'll write only columns that exist in Excel (to preserve structure)
    excel_cols_in_order = [h for h in headers if h is not None]

    for i, row in enumerate(new_df.to_dict(orient="records")):
        excel_row = start_row + i

        for h in excel_cols_in_order:
            col = header_to_col[h]

            if h == "Estado":
                # default for newly added jobs
                ws.cell(row=excel_row, column=col, value=default_estado)
            else:
                ws.cell(row=excel_row, column=col, value=row.get(h, None))

        # If Excel has Estado but header list didn't include it for some reason
        if estado_col and "Estado" not in header_to_col:
            ws.cell(row=excel_row, column=estado_col, value=default_estado)

    wb.save(FILE)
    return len(new_df)

In [110]:
FILE = "jobhunt.xlsx"

if not os.path.exists(FILE):
    # first time: create + full formatting
    jobs_df.to_excel(FILE, index=False)
    format_jobhunt_excel(FILE)
    auto_resize_columns(FILE)

else:
    # DO NOT to_excel() again — it wipes formatting
    added = append_new_jobs_preserve_format(FILE, jobs_df, id_col="job_id")
    auto_resize_columns(FILE)
    print(f"Added {added} new rows")

Added 10 new rows
